# Mini Project 2: Data Exploration and Visualisation

In [ ]:
# CELL 0 — Environment
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import shapiro

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

# Inline plots
%matplotlib inline

## 1. Paths & Basic Config

Update these paths to your local files. The rest of the notebook uses these variables.


In [ ]:
# CELL 1 — Paths
DATA_RED    = r"C:/Users/busin/Documents/bi_assignments/MP2/winequality-red.xlsx"
DATA_WHITE  = r"C:/Users/busin/Documents/bi_assignments/MP2/winequality-white.xlsx"
DATA_PUBLIC = r"C:/Users/busin/Documents/bi_assignments/MP2/WineQT.csv"

## 2. Loader & Cleaner Helpers

- `load_wine(path)` loads CSV/Excel. If the first row is the true header (shifted into data), it fixes it.  
- `clean(df)` standardises columns, coerces to numeric, drops NAs, deduplicates.

In [ ]:
# CELL 2 — Load & clean helpers
def load_wine(path: str) -> pd.DataFrame:
    """Load CSV or Excel. Fixes 'Unnamed' header rows if needed."""
    if path.lower().endswith(".csv"):
        df = pd.read_csv(path)
    else:
        df = pd.read_excel(path)
        if any(str(c).lower().startswith("unnamed") for c in df.columns):
            df.columns = df.iloc[0]
            df = df.iloc[1:].reset_index(drop=True)
    return df

def clean(df: pd.DataFrame) -> pd.DataFrame:
    """Standard clean: names, numeric coercion, dropna, deduplicate."""
    df = df.copy()
    df.columns = [str(c).strip().lower().replace(" ", "_").replace("-", "_") for c in df.columns]
    if "id" in df.columns:
        df = df.drop(columns=["id"])
    cols = [
        "fixed_acidity","volatile_acidity","citric_acid","residual_sugar","chlorides",
        "free_sulfur_dioxide","total_sulfur_dioxide","density","ph","sulphates",
        "alcohol","quality"
    ]
    existing = [c for c in cols if c in df.columns]
    df = df[existing]
    for c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna()
    df = df.drop_duplicates().reset_index(drop=True)
    return df


## 3. Load, Clean, Tag, Aggregate (Tasks 1–3)

We keep a `type` column to retain identity across sources.

In [ ]:
# CELL 3 — Load & clean
red_df_raw   = load_wine(DATA_RED)
white_df_raw = load_wine(DATA_WHITE)
pub_df_raw   = load_wine(DATA_PUBLIC)

red_df   = clean(red_df_raw)
white_df = clean(white_df_raw)
wineqt_df= clean(pub_df_raw)

red_df["type"]   = "Red"
white_df["type"] = "White"
wineqt_df["type"]= "Public Wine"

all_wine = pd.concat([red_df, white_df, wineqt_df], ignore_index=True)

print("Shapes:")
print("  Red         :", red_df.shape)
print("  White       :", white_df.shape)
print("  Public Wine :", wineqt_df.shape)
print("  ALL         :", all_wine.shape)

# CELL 4 — Quick peek (top rows)
all_wine.head()

# CELL 5 — Info
all_wine.info()

## 4. Descriptive Statistics (Tasks 5 & 7)
Grouped by `type`.

In [ ]:
# CELL 6 — Grouped describe
desc = all_wine.groupby("type").describe()
desc.columns = ["_".join(col).strip() for col in desc.columns.values]
desc.head()

In [ ]:
# CELL 7 — (Optional) Save descriptive stats
# desc.to_excel("wine_descriptive_stats.xlsx", index=True)
pass

## 5. Plotting Helpers (Reusable)

These helpers make charts consistent and easy to tweak.

In [ ]:
# CELL 8 — Plotting helpers (matplotlib only, 1 chart per call)

def plot_hist(df, col, bins=30, title=None, xlabel=None, ylabel="Frequency"):
    data = df[col].dropna()
    plt.figure(figsize=(7,4))
    plt.hist(data, bins=bins)
    plt.title(title or f"Histogram of {col}")
    plt.xlabel(xlabel or col)
    plt.ylabel(ylabel)
    plt.show()

def plot_bar_series(series, title=None, xlabel=None, ylabel=None, rot=0):
    ax = series.plot(kind="bar", figsize=(7,4))
    ax.set_title(title or "")
    ax.set_xlabel(xlabel or "")
    ax.set_ylabel(ylabel or "")
    plt.xticks(rotation=rot)
    plt.show()

def plot_crosstab_bar(df, x_col, hue_col, title=None, xlabel=None, ylabel="Count"):
    ct = pd.crosstab(df[x_col], df[hue_col]).sort_index()
    ax = ct.plot(kind="bar", figsize=(8,5))
    ax.set_title(title or f"{x_col} by {hue_col}")
    ax.set_xlabel(xlabel or x_col)
    ax.set_ylabel(ylabel)
    plt.show()

def plot_heatmap_corr(df_numeric, title="Correlation Heatmap"):
    corr = df_numeric.corr()
    plt.figure(figsize=(12,8))
    im = plt.imshow(corr, aspect="auto")
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.title(title)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.index)), corr.index)
    for (i,j), val in np.ndenumerate(corr.values):
        plt.text(j, i, f"{val:.2f}", ha="center", va="center", fontsize=7)
    plt.tight_layout()
    plt.show()

def boxplot_by_quality_and_type(df, y_col, types=("Red","White","Public Wine"), title=None):
    """One cluster per quality; within cluster one box per type (if data available)."""
    qualities = sorted(df["quality"].dropna().unique())
    fig, ax = plt.subplots(figsize=(12,5))
    pos = 1
    x_positions = []
    x_labels    = []
    for q in qualities:
        subset = df[df["quality"]==q]
        group_positions = []
        for t in types:
            arr = subset[subset["type"]==t][y_col].dropna().values
            if len(arr) > 0:
                ax.boxplot(arr, positions=[pos], widths=0.6, patch_artist=False)
                group_positions.append(pos)
                pos += 1
        pos += 1  # spacer between quality groups
        if group_positions:
            x_positions.append(np.mean(group_positions))
            x_labels.append(str(q))
    ax.set_xticks(x_positions)
    ax.set_xticklabels(x_labels)
    ax.set_xlabel("Quality")
    ax.set_ylabel(y_col)
    ax.set_title(title or f"{y_col} vs Quality by Type")
    ax.grid(axis="y", linestyle=":", alpha=0.5)
    plt.show()

## 6. Target Distribution: `quality` (Tasks 5 & 8a)

**What this shows:** Count of samples at each quality score, split by `type`.  
Use it to see class balance, popular quality levels, and type differences.

In [ ]:
# CELL 9 — Quality distribution by type
plot_crosstab_bar(all_wine, x_col="quality", hue_col="type",
                  title="Distribution of Wine Quality Scores by Type",
                  xlabel="Quality")

## 7. Histograms — Alcohol / Residual Sugar / Volatile Acidity (Task 5)

**What this shows:** The univariate distribution (shape, skew, outliers) of key predictors.  
Tip: Change `BINS` if you want finer/coarser detail.

In [ ]:
# CELL 10 — Easy bin control
BINS = 30

In [ ]:
# CELL 11 — Alcohol
plot_hist(all_wine, "alcohol", bins=BINS, title="Histogram of Alcohol")

In [ ]:
# CELL 12 — Residual Sugar
plot_hist(all_wine, "residual_sugar", bins=BINS, title="Histogram of Residual Sugar")

In [ ]:
# CELL 13 — Volatile Acidity
plot_hist(all_wine, "volatile_acidity", bins=BINS, title="Histogram of Volatile Acidity")

## 8. Correlations (Task 5)

**What this shows:** Pearson correlations between numeric features and `quality`.  
Hot/cold colors indicate positive/negative associations; annotations show coefficients.